In [1]:
import acquire, prepare, model

In [2]:
df = acquire.get_corpus()

df['clean_and_lem'] = df.readme.apply(prepare.basic_clean).apply(prepare.lemmatize)

df = prepare.cut_singles(df)

In [ ]:
#HOLD until we try it without it
##
#df['encoded_language'] = model.encode_categories(df.language)

In [9]:
X_train, X_test, train, test = model.make_model_components(df.clean_and_lem, df.language, .3)

In [10]:
predictons = model.run_the_tree_model(X_train, train.actual, 6)

In [11]:
train['predictions'] = predictons

In [12]:
model.score_your_model(train.actual, train.predictions)

Accuracy: 90.48%
---
Confusion Matrix
actual       C  C++  Go  Java  JavaScript  None  Python  Rust  TypeScript  Vue
predictions                                                                   
C            1    0   0     0           0     0       0     0           0    0
C++          0    3   0     0           0     0       0     0           0    0
Go           0    0   3     0           0     0       0     0           0    0
Java         0    1   0     6           0     2       0     0           2    1
JavaScript   0    0   0     0          24     0       0     0           0    0
None         0    0   0     0           0     7       0     0           0    0
Python       0    0   0     0           0     0      10     0           0    0
Rust         0    0   0     0           0     0       0     1           0    0
TypeScript   0    0   0     0           0     0       0     0           2    0
---
              precision    recall  f1-score   support

           C       1.00      1.00 

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
test_predictons = model.run_the_tree_model(X_test, test.actual, 6)
test['predictions'] = test_predictons

In [14]:
model.score_your_model(test.actual, test.predictions)

Accuracy: 88.89%
---
Confusion Matrix
actual       C  C++  Go  Java  JavaScript  None  Python  Rust  TypeScript  Vue
predictions                                                                   
C            1    0   0     0           0     0       0     0           0    0
C++          0    2   0     0           0     0       0     0           0    0
Go           0    0   1     0           0     0       0     1           1    1
Java         0    0   0     2           0     0       0     0           0    0
JavaScript   0    0   0     0          10     0       0     0           0    0
None         0    0   0     0           0     3       0     0           0    0
Python       0    0   0     0           0     0       4     0           0    0
TypeScript   0    0   0     0           0     0       0     0           1    0
---
              precision    recall  f1-score   support

           C       1.00      1.00      1.00         1
         C++       1.00      1.00      1.00         2
     

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
df2 = acquire.get_corpus()

df2['clean-lem-no_stop'] = df2.readme.apply(prepare.basic_clean).apply(prepare.remove_stopwords).apply(prepare.lemmatize)

df2 = prepare.cut_singles(df2)

In [18]:
df2.head()

,language,readme,repo,clean-lem-no_stop
0,JavaScript,\n\n\n\n\n\nfreeCodeCamp.org's open-source cod...,freeCodeCamp/freeCodeCamp,freecodecamporg's opensource codebase curricul...
1,Rust,\n996.ICU\nPlease note that there exists NO ot...,996icu/996.ICU,996icu please note exists official account app...
2,JavaScript,\n\n\n\n\n\n\n\n\n\n\n\nSupporting Vue.js\nVue...,vuejs/vue,supporting vuejs vuejs mitlicensed open source...
3,JavaScript,\nReact · \nReact is a JavaScript library f...,facebook/react,react react javascript library building user i...
4,C++,\n\n\n\n\n\n\nDocumentation\n\n\n\n\n\n\n\n\nT...,tensorflow/tensorflow,documentation tensorflow endtoend open source ...


In [21]:
X_train, X_test, train, test = model.make_model_components(df2['clean-lem-no_stop'], df2.language)

In [23]:
predictons2 = model.run_the_tree_model(X_train, train.actual, 6)

In [24]:
train['predictions'] = predictons2

In [25]:
model.score_your_model(train.actual, train.predictions)

Accuracy: 83.33%
---
Confusion Matrix
actual       C  C++  Go  Java  JavaScript  None  Python  Rust  TypeScript  Vue
predictions                                                                   
C            2    2   2     1           2     1       1     1           0    2
C++          0    3   0     0           0     0       0     0           0    0
Go           0    0   1     0           0     0       0     0           0    0
Java         0    0   0     5           0     0       0     0           0    0
JavaScript   0    0   0     0          25     0       0     0           0    0
None         0    0   0     0           0     8       0     0           0    0
Python       0    0   0     0           0     0      10     0           0    0
Rust         0    0   0     0           0     0       0     1           0    0
TypeScript   0    0   0     0           0     0       0     0           5    0
---
              precision    recall  f1-score   support

           C       0.14      1.00 

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
test_predictons2 = model.run_the_tree_model(X_test, test.actual, 6)
test['predictions'] = test_predictons2

In [27]:
model.score_your_model(test.actual, test.predictions)

Accuracy: 100.00%
---
Confusion Matrix
actual       C++  Go  Java  JavaScript  None  Python  TypeScript
predictions                                                     
C++            1   0     0           0     0       0           0
Go             0   1     0           0     0       0           0
Java           0   0     2           0     0       0           0
JavaScript     0   0     0           7     0       0           0
None           0   0     0           0     3       0           0
Python         0   0     0           0     0       3           0
TypeScript     0   0     0           0     0       0           1
---
              precision    recall  f1-score   support

         C++       1.00      1.00      1.00         1
          Go       1.00      1.00      1.00         1
        Java       1.00      1.00      1.00         2
  JavaScript       1.00      1.00      1.00         7
        None       1.00      1.00      1.00         3
      Python       1.00      1.00      1.00    

In [1]:
from requests import get
from bs4 import BeautifulSoup

import pandas as pd

import json
import os

In [8]:
def scrape_links_one_page(url):
    headers = {'User-Agent': 'Codeup Data Science'} 
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    extension = soup.find_all("a", {"class":"v-align-middle"}, {"data-hydro-click":"url"})
    extensions = []
    for count in range(len(extension)):
        extensions.append("http://github.com/" + extension[count].get_text())
    return extensions

def make_url_list():
    """
    Creates a 10 page list. Each page needs the links to repositories scapes.
    """
    urls = []
    for count in range(11,21):
        urls.append("https://github.com/search?p={}&q=stars%3A%3E100&type=Repositories".format(count))
    return urls


def loop_through_urls(url_list):
    """
    Creates a 100 page list. Each element is a link to be scraped of their README.
    """
    big_list = []
    for url in url_list:
        big_list.extend(scrape_links_one_page(url))
    return big_list

def get_readme_text(url):
    response = get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    textboxes = soup.findAll("div", {"class": "Box-body"})
    readme_text = textboxes[0].text

    language = soup.find("span", {"class": "lang"})
    if language == None:
        language = "None"
    else:
        language = soup.find("span", {"class": "lang"}).get_text()

    return {
        "repo" : url[18:],
        "language" : language,
        "readme" : readme_text
    }


In [9]:
url_list = make_url_list()

In [10]:
url_list

['https://github.com/search?p=11&q=stars%3A%3E100&type=Repositories',
 'https://github.com/search?p=12&q=stars%3A%3E100&type=Repositories',
 'https://github.com/search?p=13&q=stars%3A%3E100&type=Repositories',
 'https://github.com/search?p=14&q=stars%3A%3E100&type=Repositories',
 'https://github.com/search?p=15&q=stars%3A%3E100&type=Repositories',
 'https://github.com/search?p=16&q=stars%3A%3E100&type=Repositories',
 'https://github.com/search?p=17&q=stars%3A%3E100&type=Repositories',
 'https://github.com/search?p=18&q=stars%3A%3E100&type=Repositories',
 'https://github.com/search?p=19&q=stars%3A%3E100&type=Repositories',
 'https://github.com/search?p=20&q=stars%3A%3E100&type=Repositories']

In [13]:
big_list = loop_through_urls(url_list)

In [18]:
pages10_20 = pd.Series(big_list)

In [19]:
pages10_20

0                    http://github.com/antirez/redis
1                    http://github.com/denoland/deno
2                    http://github.com/gohugoio/hugo
3                 http://github.com/ionic-team/ionic
4     http://github.com/google/material-design-icons
                           ...                      
85     http://github.com/Trinea/android-open-project
86          http://github.com/PhilJay/MPAndroidChart
87    http://github.com/tiimgreen/github-cheat-sheet
88                   http://github.com/XX-net/XX-Net
89         http://github.com/sahat/hackathon-starter
Length: 90, dtype: object

In [20]:
def make_corpus(big_list):
    corpus = [get_readme_text(url) for url in big_list]
    return corpus

In [21]:
corpus2 = make_corpus(big_list)

In [23]:
df_corpus2 = pd.DataFrame(corpus2)

In [24]:
df_corpus2

,repo,language,readme
0,antirez/redis,C,\nThis README is just a fast quick start docum...
1,denoland/deno,TypeScript,\nDeno\nA secure runtime for JavaScript and Ty...
2,gohugoio/hugo,Go,\n\nA Fast and Flexible Static Site Generator ...
3,ionic-team/ionic,TypeScript,\nIonic\nIonic is the open-source mobile app d...
4,google/material-design-icons,CSS,\nMaterial design icons\nMaterial design icons...
...,...,...,...
85,Trinea/android-open-project,None,\nAndroid 开源项目分类汇总，更全更新可见 codekk.com\nOther: E...
86,PhilJay/MPAndroidChart,Java,\n\n\n\n\n\n\n⚡️ A powerful & easy to use char...
87,tiimgreen/github-cheat-sheet,None,\nGitHub Cheat Sheet \nA collection of cool hi...
88,XX-net/XX-Net,Python,\nXX-Net\n中文文档 English فارسی صفحه اص...


In [ ]:
def get_corpus():
    filename = 'data_second.json'

    # check for presence of the file or make a new request
    if os.path.exists(filename):
        with open(filename) as json_file:
            reads = json.load(json_file)
        return pd.DataFrame(reads)
    else:
        data = make_corpus()
        with open(filename, 'w') as json_file:
            json.dump(data, json_file)
        return pd.DataFrame(data)

In [25]:
filename = 'data_second.json'

In [26]:
with open(filename, 'w') as json_file:
    json.dump(df_corpus2, json_file)

TypeError: Object of type DataFrame is not JSON serializable